In [31]:
import mysql.connector
import numpy as np
import pandas as pd
import requests 
import yfinance as yf
creds = {
    'host' : "localhost",
    'user' : "root",
     'password' : "n3u3da!",
     'database' : "CSFoundations"
}


In [13]:
res= requests.get("http://127.0.0.1:5000/api/v1/stock/AAPL/history")

In [52]:
tickers= ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'NFLX', 'FB', 'NVDA', 'DIS', 'V']
# data= []
# for ticker in tickers:
#             stock = yf.Ticker(ticker)
#             info = stock.info

#             # calculating the price growth
#             current = info.get("regularMarketPrice", 0)
#             previous = info.get("previousClose", 1) 
#             growth = (current - previous) / previous

#             # stock info
#             data.append({
#                 "ticker": ticker,
#                 "name": info.get("shortName", ""),
#                 "price": current,
#                 "previous_close": previous,
#                 "growth": growth
#             })
# data            

In [53]:
stocks = yf.Tickers(tickers).download(period='3mo')




[*********************100%***********************]  10 of 10 completed


In [ ]:
price_history= stocks.Close

Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2025-05-07,195.992981,188.710007,151.196960,432.555908,276.220001
2025-05-08,197.231369,192.080002,154.093460,437.367065,284.820007
2025-05-09,198.270004,193.059998,152.565308,437.926056,298.260010
2025-05-12,210.789993,208.639999,158.268402,448.436737,318.380005
2025-05-13,212.929993,211.369995,159.337097,448.316986,334.070007
...,...,...,...,...,...
2025-07-31,207.570007,234.110001,191.899994,533.500000,308.269989
2025-08-01,202.380005,214.750000,189.130005,524.109985,302.630005
2025-08-04,203.350006,211.649994,195.039993,535.640015,309.260010


In [39]:
dir(stocks)

['Close',
 'Dividends',
 'High',
 'Low',
 'Open',
 'T',
 'Volume',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__dataframe_consortium_standard__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__firstlineno__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',

In [63]:
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor(dictionary=True)
    cursor.execute("select ticker, quantity, transaction_date from stockdemo order by transaction_date desc")
    history = cursor.fetchall()

In [ ]:
ticker_and_quantity = pd.DataFrame(history, columns=['ticker', 'quantity', 'transaction_date'])
ticker_and_quantity['transaction_date'] = pd.to_datetime(ticker_and_quantity['transaction_date'])
ticker_and_quantity= ticker_and_quantity.set_index('transaction_date')
ticker_and_quantity = ticker_and_quantity.sort_index()
ticker_and_quantity['quantity'] = ticker_and_quantity[ticker_and_quantity.index].groupby(['ticker'])['quantity'].cumsum()
# ticker_and_quantity['quantity'] = ticker_and_quantity['quantity'].astype(float)
# ticker_and_quantity = ticker_and_quantity.set_index('ticker')

In [50]:
#Input fake entries to stock database
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor()
    #Create dummy entries for teh portfolio
    stockinfo = [("AAPL", "Apple", "150.20", "21"),
                    ("GOOGL", "Alphabet", "280.50", "15"),
                    ("AMZN", "Amazon", "34.00", "12"),
                    ("MSFT", "Microsoft", "2.50", "1"),
                    ("TSLA", "Tesla", "750.00", "9"),
                    ("NFLX", "Netflix", "550.00", "40"),
                    ("FB", "Facebook", "350.00", "5"),
                    ("NVDA", "Nvidia", "60.00", "80"),
                    ("DIS", "Disney", "80.00", "5"),
                    ("V", "Visa", "20.00", "1")]
    #Insert the dummy entries into the stock table
    cursor.executemany("INSERT INTO stockdemo (ticker, stock_name, stock_value, quantity) VALUES (%s, %s, %s, %s)", stockinfo)
    #Commit the changes to the database 
    stock_db.commit()